Hey!

TL;DR: **run the whole notebook** :)

Enter your ClientID and CLient secret when asked.

Down below, an object of a custom class `SongCollector()` will be created, which has a function `collect_songs_from_playlists(<playlist_ids>, remove_duplicates={True|False})`

An object of that collector is then created in a variable called `my_songcollector` and its function `.collect_songs_from_playlists(playlist_ids)` is executed for a list of `playlist_ids`

The process will then run through (takes maybe a minute or so), and you can easily extract the dataframe out of 
`my_songcollector` with

```python
my_songcollector.df
```

The class is pretty hardcoded, there are probably more elegant solutions ;)

In [2]:
import json
from IPython.display import Image

import pandas as pd
from tqdm.notebook import tqdm

# Importing spotify API stuff
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import getpass
import time
import random
import requests

import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns

# Machine learning
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_samples, silhouette_score

# Import my custom modules in the same folder (saved as .py files)
# from scrape_billboard import scrape_billboard
# import recommender_mvp


In [3]:
client_id = getpass.getpass(prompt="Spotify client_id: ")
client_secret = getpass.getpass(prompt="Spotify client_secret: ")

#Initialize a SpotiPy instance with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret)
                    )

Spotify client_id: ········
Spotify client_secret: ········


In [4]:
#Function to get the artists involved in a song:

def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [16]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def create_df_from_tracks(tracks):
    # making sure we only get track who really have an id
    tracks = [track for track in tracks if track_exists(track) and id_exists(track)]
    #tracks = [track for track in tracks if id_exists(track)]
    
    # getting all the ids from the tracks
    ids = [get_id(track) for track in tracks]
    
    # Getting the song features from the song ids
    # Warning: Only 100 tracks here
    
    features = sp.audio_features(ids)
    
    try:
        df = pd.DataFrame({'name': [track['track']['name'] for track in tracks],
                           # a track can have mulitple artists. if so, add them seperated by ","
                           'artists': [", ".join([artist['name'] for artist in track['track']['artists']]) for track in tracks],
                           # 'ids': ids,
                          })
        
        # Creating a dataframe from the features 
        df_features = pd.DataFrame(features)
        
        return pd.concat([df, df_features], axis=1)
    
    except AttributeError:
        return pd.DataFrame(columns=['name',
                                     'danceability',
                                        'energy',
                                        'key',
                                        'loudness',
                                        'mode',
                                        'speechiness',
                                        'acousticness',
                                        'instrumentalness',
                                        'liveness',
                                        'valence',
                                        'tempo',
                                        'type',
                                        'id',
                                        'uri',
                                        'track_href',
                                        'analysis_url',
                                        'duration_ms',
                                        'time_signature'],
                           )

def get_id(track):
        return track['track']['id']
    
def id_exists(track):
    return bool(get_id(track))

def track_exists(track):
    return track['track']

In [6]:
import pandas as pd

# The Class - which is the blueprint of the robot. You can build multiple robots with different 
# attributes from it
class SongCollector:
    
    # the "constructor" method. These things should happen, when our robot is created
    def __init__(self, dataframe=pd.DataFrame()):
        # "self", the "identity" or consciousness of your individual robot.
        # in this case we say that when the user wants to create the robot with a song dataframe already 
        # in its pocket, the user can initialize it with that and it gets added to the robots "identity"
        self.df = dataframe
        
    # one "ability" of the robot
    def collect_songs_from_playlists(self,
                                     playlist_ids,
                                     remove_duplicates=False):
        """
        Takes the dataframe stored in self.df and expands it by the songs 
        provided by the user in the list <playlists>
        
        playlists: A list of spotify playlist URIs as strings
        """
        for playlist_id in tqdm(playlist_ids):
            tracks = get_playlist_tracks(playlist_id)
            try:
                playlist_df = create_df_from_tracks(tracks)
                self.df = pd.concat([self.df, playlist_df], axis=0)
            except (requests.HTTPError, spotipy.client.SpotifyException) as exception:
                # print(exception)
                continue
            
        # the robot should return the resulting dataframe
        return self.df

In [17]:
# check the 'country' or the 'locale' keyword in categories() to customize your playlist selection 
categories = sp.categories()

In [18]:
categories_dct = {cat['name']: cat['id'] for cat in categories['categories']['items']}
categories_dct

{'Top Lists': 'toplists',
 'At Home': 'at_home',
 'Pop': 'pop',
 'Mood': 'mood',
 'Decades': 'decades',
 'Hip Hop': 'hiphop',
 'In the car': 'in_the_car',
 'Gaming': 'gaming',
 'Wellness': 'wellness',
 'Workout': 'workout',
 'Chill': 'chill',
 'Focus': 'focus',
 'Sleep': 'sleep',
 'Party': 'party',
 'Indie': 'indie_alt',
 'Metal': 'metal',
 'Rock': 'rock',
 'Dance/Electronic': 'edm_dance',
 'Cooking & Dining': 'dinner',
 'Jazz': 'jazz'}

In [19]:
# and for all
playlist_ids = []

for cat_id in categories_dct.values():
    pl_per_categeroy_json = sp.category_playlists(cat_id)['playlists']['items']
    
    print(pl_per_categeroy_json)
    for pl in pl_per_categeroy_json:
        try:
            id_ = pl['id']
        except TypeError:
            continue
        playlist_ids.append(f"spotify:playlist:{id_}")


my_songcollector = SongCollector()

my_songcollector.collect_songs_from_playlists(playlist_ids)

[{'collaborative': False, 'description': "The UK's biggest playlist. Cover: Lil Tjay & 6LACK", 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWY4lFlS4Pnso'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWY4lFlS4Pnso', 'id': '37i9dQZF1DWY4lFlS4Pnso', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f00000003c500d02e058173099ad2a4c7', 'width': None}], 'name': 'Hot Hits UK', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTYxNDI0OTY4MCwwMDAwMDM2MjAwMDAwMTc3ZDhjNzVlNWUwMDAwMDE3N2I2MTAzMDBk', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWY4lFlS4Pnso/tracks', 'total': 70}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DWY4lFlS4Pnso'}, {'collaborative': False, 'description': 'The

[{'collaborative': False, 'description': "Get happy with today's dose of feel-good songs!", 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX3rxVfibe1L0'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX3rxVfibe1L0', 'id': '37i9dQZF1DX3rxVfibe1L0', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f00000003aa93fe4e8c2d24fc62556cba', 'width': None}], 'name': 'Mood Booster', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTYxNDI1OTUzOCwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX3rxVfibe1L0/tracks', 'total': 76}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DX3rxVfibe1L0'}, {'collaborative': False, 'description': 'Feel 

[{'collaborative': False, 'description': 'Smart, engaging podcasts that go beyond the headlines.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX0BxHamIEkKV'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0BxHamIEkKV', 'id': '37i9dQZF1DX0BxHamIEkKV', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f000000036508700a5814ac32367b1967', 'width': None}], 'name': 'Stay in the Know', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTYxNDAxMTgxMSwwMDAwMDAyMDAwMDAwMTc3Y2E5OWM4NzYwMDAwMDE3Mjc0NTI5YWE4', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0BxHamIEkKV/tracks', 'total': 5}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DX0BxHamIEkKV'}, {'collaborative': False, 'descriptio

[{'collaborative': False, 'description': 'Get your beast mode on!', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX76Wlfdnj7AP'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX76Wlfdnj7AP', 'id': '37i9dQZF1DX76Wlfdnj7AP', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f000000039249b35f23fb596b6f006a15', 'width': None}], 'name': 'Beast Mode', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTYxNDI1OTUzOCwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX76Wlfdnj7AP/tracks', 'total': 200}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DX76Wlfdnj7AP'}, {'collaborative': False, 'description': 'Pop hits to keep your workout 

[{'collaborative': False, 'description': 'Kick back to the best new and recent chill tunes.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX4WYpdgoIcn6'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4WYpdgoIcn6', 'id': '37i9dQZF1DX4WYpdgoIcn6', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f00000003cf8e264c6a92e245402ecb7a', 'width': None}], 'name': 'Chill Hits', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTYxNDI1OTUzOCwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4WYpdgoIcn6/tracks', 'total': 130}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DX4WYpdgoIcn6'}, {'collaborative': False, 'description': 'Just

...

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5ghIJDpPoe3CfHMGu71E6T,2IZZqH4K02UIYg5EohpNHF,3d9DChrdc6BOeFsbrZ3Is0,59WN2psjkt1tyaxjspN8fp,5wj4E6IsrVtn8IBJQOd0Cl,1hKdDCpiI9mqz1jVHRKG0E,4P5KoWXOxwuobLmHXLMobV,5yEPxDjbbzUzyauGtnmVEC,2EoOZnxNgtmZaD8uUmz2nD,1FTSo4v6BOZH9QxKc3MbVM,4d9RTWdrFLVAGhdzvqxkwn,5QLHGv0DfpeXLNFo7SFEy1,2TAQ9YGehOKWDqDak5DuXc,7yCPwWs66K8Ba5lFuU2bcx,3HfB5hBU0dmBt8T0iCmH42,5PntSbMHC1ud6Vvl8x56qd,6urCAbunOQI4bLhmGpX7iS,1Dr1fXbc2IxaK1Mu8P8Khz,1FkoVC85Ds3mFoK0fVqEqP,5jafMI8FLibnjkYTZ33m0c,7nnWIPM5hwE3DaUBkvOIpy,62nQ8UZVqR2RMvkJHkcO2o,4YPhn26bIFm2KUkL1VLzQG,0NJC0FDCODpPUntRTTQq97,7lksFyVBmmkYBTVQcxnvC3,0uppYCG86ajpV2hSR3dJJ0,3YRCqOhFifThpSRFJ1VWFM,3LRJbFT9rKoKv4aW7PuBJC,39tLc4Xp58Lu4KcWHggeE2,45Ia1U4KtIjAPPU7Wv1Sea,2lXHg7BtTNoP39AezyCSfL,1HZ3cUZUw5htSFmah1V8Ko,7CVYxHq1L0Z4G84jTDS6Jl,0Puj4YlTm6xNzDDADXHMI9,1Z5LCXu5Ca3sHAy6OdUrBT,5G0vAvojJAb8np3INdsISs,2V4Bc2I962j7acQj1N0PiQ,6txWz9UapYHVxEd7dDIHXT,05NYcsjJwOYq4jIiKPVj9p,6mfkHv00I6jhHIn7EWHJ2N,6cJqlSO19vs

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=00JYHDGMCHtMnCw3iqBxkY,7KxFCQHFflx0ZlY9SPT5nW,3ik11I4X2Mr3QMUPg1W1xU,3EAwcVzkJyIp481tAtYeR0,1Uwvfrav6kkJUWHECeokme,5CWGRrNRxRo0KXhMJuMm1S,5H0zLjf4O81HbJBa4FHOO9,0yjxW8s0Uuk9uKi0Nh9B7T,5KGn7C4GflN0EFMNqDmUY9,0Hq59Tzh4STCnCdokgbqJb,3ms9tJvFtCRqWMagyOnUAt,7clsGP3ea7rhBaEK9ga898,3DbvmrRTdEmnH5AzoxMp6m,6bVWGtN2BVH53il4JSFON8,75xY9IS7nE2da6q4207HsW,52rihGF8IZ6taYomWCTt5a,0WAgwdyjTwx2TT55W8z8lu,4cFeXui0uAhC2f4fTwN4P7,2J9DxFYDgET6jdSfh3HbKx,1dQtt3K8GyPmH7ZD1LJ2ox,5krfM2qSv73Km5MMgzfkBJ,3vIPdMmbsye0YMhXy4GeXt,3SXRZuCEGbbxsPiHyqwWuG,2GvnieNPXvvgnrQne1HEtl,4GtsU8mY7yrjwOb0QkCJ6i,7jorOCfCLGKRdp2F1zKLP4,4crczEC1PpoD8TsDsmoWNI,0id7TrBjYYzQ5v5mHWTUxC,7hSTXrC5oAMss4LTPbbkua,2iyVwlrSE1jJOrohg5tv0z,133YafRPaKLYCR28FSbN4M,5YaskwnGDZFDRipaqzbwQx,5k1uIPpsHmnbWFr4nbpdYb,1w8DxFHDDInmn5TovLKKrW,00JoqMc5KZFLpdrtymo6f0,5kGVL2l2eEqaFPuYU1mdbl,7GiozRoMk95aFl1WbrDdjX,1bmvJkAA8Yz9bv6y3WOj3U,5W05N4aETUauCVzddI8L66,074x9OaRq8m4Kn3J3Qgavf,6KpGyJIyA75

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=11RXJ1AauEmSX75iakdL21,0lRfksu6CZrxY8LrsIsHjd,5loBw8PvbyUDNhagi4HE7S,6QnLtJfGV9lLLuQU59Knfe,1HXGuxXMP8xWb2uQuTJBQb,0smVq2gz7xt42BXwtb2MYg,1nst5BRI3Ui94BplQ6wYjC,0qJkYnXVlPCdi3lgfhMVtW,0c6OzDThKQjNr8og0Z3hlo,79QdN1q9LGdz2FJ7I2aMW5,7txCnEZ8El270pM2aUMHRO,5cVnYm1IDWhHnDSZrWhIAn,7z9hOSVJaSVwBb0kkECwF5,1qnpuN6ktoFWsN3aF8lt6S,3TURYnoco264nSh6LN8FUF,4jUdV7tQ4gm4nsG7nItWKy,2hPZAGWvtFZ7a2Rc8YRS7J,5uV4uilxEaddomtO2BeHH1,6bQ20SLqyuVb2SdDyw15h3,3syhHFJJwtMylMJ83aoHnJ,3ik11I4X2Mr3QMUPg1W1xU,7cc54OMAnrOl7TvKgEB013,44xcyZkeyuoDivY2nuNPSM,5o990tKGmOyNWaSwRKpCay,0C4RDeGx5pEsPB4e0skgZw,7jzcImgU2sCS2TuVyKvhCX,1i4H8dTR1NQ41wYgpFXZtG,5JgW8g5qxWTEFzg50PiUqv,6tiORra6Cem9D7ljkYGc8i,5T5ik7kItjz9apmIJR7tcC,7nfwhn5s2UpTXBYxWNnOqH,6b10lRmL2g8TCAT0AdxQAX,74nXamouGrGIVARKJaSQtO,3aeGvR8wPAjuLm5xOWwojZ,41NEHLGt9hJul5U0IBad8b,6moWHXg4G0vb3v9Fd4PaD7,5ruzGMgLm1Avc5m9QGuizD,0g4EDskywvl4mFQxffm3sU,6wOtpwOUW6jsiCykEQXNEq,7wTGQKmQoj7BnjNEvJOyTe,0suGdYSCx2f

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4OKFg2NTbnszvbvl99h2zG,5UBIUZYOxM8dOf33yNq8Ui,7uNrNo3FwnPZm9aiu64oIN,2MFkg5GzZyRZKnVi3CciBw,16jl9M4H9el05evoJ4fCU4,0vjHiir9lHpZIIrvy3xWPZ,4sVY4JmPEBLPsX5GRH8S6u,2oMDxIp1I6uNQkexFBjJWQ,7wivunAmLKAfZ9j2fQrt31,6n7lMiUHBZeSOAZvPCtTIT,7tRv23Uvd3lhucDZ0OwVHI,0hdeBcZl6rdXSScMS6prYJ,0lJkobiOBH14Vw9Jznljz2,2iLdQOYdnc7sjBnGPb6vfZ,13eDtHVxDCSDSIUv06kHZq,5DFV8QllfkIoItVL6vP4AN,2m52ujxoHXXOzYoWJV3zsa,0uqwYjMkEcAAiPwSTWxzH7,6bTiKb47ao9YoIvw4vyes2,49aPuTO8KQan7jSEoK9Z6V,65akmuRiBnvX9qQoEwUT4f,59JkOkb5dZeluV2s7vhJRD,7rhoPowNAOMG9GPzZUCQD9,6gLqClAThZloqPX80vrLEb,30865hRUW10qtHqSvO6MAJ,23NGeIYW5sdpeWs2mZJKTh,6smNHgmjTDNJWaovT0NbRR,3cSFlobH6j9dGEAoBmk7YO,6QCDeEAy58PKoRwmqscuZ9,2Dr3VX8ZJASzeFyHcFeQHZ,1r596vXMEybBkkTiNJmpTk,1W2z4uJzZuA0r0MDQWJvex,2Le0PBcwBEnSeH1J2DYUBI,5cCU1vF7kwZDveCsSeEFSO,22ZVUAqhmJr68Fu2BYw3dt,7bwF8qvBd0DAquB1DVNSfV,1ZhoBg848AQ8KI8Oi7iMHc,2WMtVdi53gSyrk4KzBW5qh,3hNBfFtEyb4qFhaSLrvQi2,1lcOX9scuEaFjOuLSbpLpr,02NPmYFlgDP

,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,0
0,Get Out My Head,Shane Codd,0.693,0.9130,0,-3.035,1,0.0298,0.0542,0.000069,...,0.7950,123.989,audio_features,055Mme9ReKK99jRFA5UJ55,spotify:track:055Mme9ReKK99jRFA5UJ55,https://api.spotify.com/v1/tracks/055Mme9ReKK9...,https://api.spotify.com/v1/audio-analysis/055M...,204933,4,NaN
1,Don't Play,"Anne-Marie, KSI, Digital Farm Animals",0.695,0.7840,6,-6.564,0,0.0402,0.0889,0.000000,...,0.5200,128.031,audio_features,4I5bvu2KDsrCg0EWHIcvul,spotify:track:4I5bvu2KDsrCg0EWHIcvul,https://api.spotify.com/v1/tracks/4I5bvu2KDsrC...,https://api.spotify.com/v1/audio-analysis/4I5b...,188581,4,NaN
2,WITHOUT YOU,The Kid LAROI,0.662,0.4130,0,-7.357,1,0.0299,0.2130,0.000000,...,0.4670,93.005,audio_features,27OeeYzk6klgBh83TSvGMA,spotify:track:27OeeYzk6klgBh83TSvGMA,https://api.spotify.com/v1/tracks/27OeeYzk6klg...,https://api.spotify.com/v1/audio-analysis/27Oe...,161385,4,NaN
3,Friday (feat. Mufasa & Hypeman) - Dopamine Re-...,"Riton, Nightcrawlers, Mufasa & Hypeman, Dopamine",0.824,0.8620,2,-3.424,1,0.1260,0.0076,0.000132,...,0.8010,122.980,audio_features,4cG7HUWYHBV6R6tHn1gxrl,spotify:track:4cG7HUWYHBV6R6tHn1gxrl,https://api.spotify.com/v1/tracks/4cG7HUWYHBV6...,https://api.spotify.com/v1/audio-analysis/4cG7...,169153,4,NaN
4,drivers license,Olivia Rodrigo,0.585,0.4360,10,-8.761,1,0.0601,0.7210,0.000013,...,0.1320,143.874,audio_features,7lPN2DXiMsVn7XUKtOW1CS,spotify:track:7lPN2DXiMsVn7XUKtOW1CS,https://api.spotify.com/v1/tracks/7lPN2DXiMsVn...,https://api.spotify.com/v1/audio-analysis/7lPN...,242014,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,"Nocturne No. 20 in C-sharp minor, Op. posthume","Frédéric Chopin, Jacques Loussier",0.485,0.1350,6,-18.958,0,0.0357,0.9910,0.907000,...,0.2100,117.079,audio_features,7btkqjF0Jw2DopaH6wvgvD,spotify:track:7btkqjF0Jw2DopaH6wvgvD,https://api.spotify.com/v1/tracks/7btkqjF0Jw2D...,https://api.spotify.com/v1/audio-analysis/7btk...,116743,3,NaN
50,"Beethoven's Piano Sonata No. 8, Pathetique",Hiromi,0.548,0.0955,1,-22.531,1,0.0368,0.7410,0.930000,...,0.0594,129.332,audio_features,7hdbfMdjsdclkZONmT7lD6,spotify:track:7hdbfMdjsdclkZONmT7lD6,https://api.spotify.com/v1/tracks/7hdbfMdjsdcl...,https://api.spotify.com/v1/audio-analysis/7hdb...,313027,4,NaN
51,Boléro,Trio X of Sweden,0.553,0.3540,0,-15.213,1,0.0433,0.7570,0.881000,...,0.6820,79.314,audio_features,0okBlGJdxmDos1f6H4PyRq,spotify:track:0okBlGJdxmDos1f6H4PyRq,https://api.spotify.com/v1/tracks/0okBlGJdxmDo...,https://api.spotify.com/v1/audio-analysis/0okB...,431787,3,NaN
52,First Gymnopedie,Yusef Lateef,0.449,0.3600,7,-17.882,1,0.2100,0.9920,0.888000,...,0.1930,78.916,audio_features,6A5DjTiYSkelNb2QuEV4vt,spotify:track:6A5DjTiYSkelNb2QuEV4vt,https://api.spotify.com/v1/tracks/6A5DjTiYSkel...,https://api.spotify.com/v1/audio-analysis/6A5D...,207440,3,NaN


In [20]:
# run this to get returned the dataframe of songs with their audiofeatures that were collected
my_songcollector.df

,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,0
0,Get Out My Head,Shane Codd,0.693,0.9130,0,-3.035,1,0.0298,0.0542,0.000069,...,0.7950,123.989,audio_features,055Mme9ReKK99jRFA5UJ55,spotify:track:055Mme9ReKK99jRFA5UJ55,https://api.spotify.com/v1/tracks/055Mme9ReKK9...,https://api.spotify.com/v1/audio-analysis/055M...,204933,4,NaN
1,Don't Play,"Anne-Marie, KSI, Digital Farm Animals",0.695,0.7840,6,-6.564,0,0.0402,0.0889,0.000000,...,0.5200,128.031,audio_features,4I5bvu2KDsrCg0EWHIcvul,spotify:track:4I5bvu2KDsrCg0EWHIcvul,https://api.spotify.com/v1/tracks/4I5bvu2KDsrC...,https://api.spotify.com/v1/audio-analysis/4I5b...,188581,4,NaN
2,WITHOUT YOU,The Kid LAROI,0.662,0.4130,0,-7.357,1,0.0299,0.2130,0.000000,...,0.4670,93.005,audio_features,27OeeYzk6klgBh83TSvGMA,spotify:track:27OeeYzk6klgBh83TSvGMA,https://api.spotify.com/v1/tracks/27OeeYzk6klg...,https://api.spotify.com/v1/audio-analysis/27Oe...,161385,4,NaN
3,Friday (feat. Mufasa & Hypeman) - Dopamine Re-...,"Riton, Nightcrawlers, Mufasa & Hypeman, Dopamine",0.824,0.8620,2,-3.424,1,0.1260,0.0076,0.000132,...,0.8010,122.980,audio_features,4cG7HUWYHBV6R6tHn1gxrl,spotify:track:4cG7HUWYHBV6R6tHn1gxrl,https://api.spotify.com/v1/tracks/4cG7HUWYHBV6...,https://api.spotify.com/v1/audio-analysis/4cG7...,169153,4,NaN
4,drivers license,Olivia Rodrigo,0.585,0.4360,10,-8.761,1,0.0601,0.7210,0.000013,...,0.1320,143.874,audio_features,7lPN2DXiMsVn7XUKtOW1CS,spotify:track:7lPN2DXiMsVn7XUKtOW1CS,https://api.spotify.com/v1/tracks/7lPN2DXiMsVn...,https://api.spotify.com/v1/audio-analysis/7lPN...,242014,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,"Nocturne No. 20 in C-sharp minor, Op. posthume","Frédéric Chopin, Jacques Loussier",0.485,0.1350,6,-18.958,0,0.0357,0.9910,0.907000,...,0.2100,117.079,audio_features,7btkqjF0Jw2DopaH6wvgvD,spotify:track:7btkqjF0Jw2DopaH6wvgvD,https://api.spotify.com/v1/tracks/7btkqjF0Jw2D...,https://api.spotify.com/v1/audio-analysis/7btk...,116743,3,NaN
50,"Beethoven's Piano Sonata No. 8, Pathetique",Hiromi,0.548,0.0955,1,-22.531,1,0.0368,0.7410,0.930000,...,0.0594,129.332,audio_features,7hdbfMdjsdclkZONmT7lD6,spotify:track:7hdbfMdjsdclkZONmT7lD6,https://api.spotify.com/v1/tracks/7hdbfMdjsdcl...,https://api.spotify.com/v1/audio-analysis/7hdb...,313027,4,NaN
51,Boléro,Trio X of Sweden,0.553,0.3540,0,-15.213,1,0.0433,0.7570,0.881000,...,0.6820,79.314,audio_features,0okBlGJdxmDos1f6H4PyRq,spotify:track:0okBlGJdxmDos1f6H4PyRq,https://api.spotify.com/v1/tracks/0okBlGJdxmDo...,https://api.spotify.com/v1/audio-analysis/0okB...,431787,3,NaN
52,First Gymnopedie,Yusef Lateef,0.449,0.3600,7,-17.882,1,0.2100,0.9920,0.888000,...,0.1930,78.916,audio_features,6A5DjTiYSkelNb2QuEV4vt,spotify:track:6A5DjTiYSkelNb2QuEV4vt,https://api.spotify.com/v1/tracks/6A5DjTiYSkel...,https://api.spotify.com/v1/audio-analysis/6A5D...,207440,3,NaN


In [21]:
dir(my_songcollector)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'collect_songs_from_playlists',
 'df']